### Your name: Lilian Li

<pre> Enter your name here</pre>

### Collaborators:

<pre> Enter the name of the people you worked with if any</pre>


In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Markdown reference can be found here:
    http://nestacms.com/docs/creating-content/markdown-cheat-sheet

Q1. Why would it be a problem if our training set and test set are the same.

<pre> If the training set and the test set is the same, the model will always be "high performing". This is not representative of the actual predictivity of the model accross the population. We would also not be able to identify the severity of overfitting.</pre>

Q2 [OPTIONAL]. Explain step by step the process to select k in the k-nearest neighbor algorithm (pseudocode) 

<pre> assuming we have labels in our data:
        #let dataset be an n dimension dataset
        #let metric be an already defined function to calculate distances for an n-dimensional data set
        #let estimator be an already defined function to calculate the knn approximation for the label
        #let label be the value/vector we are trying to predict using knn 
        
        trainingset = random_subset(data, num = round(nrow(data)*0.80)) #assuming we want 80% of data to be training
        testingset = remaining data
        #further subsetting for cross validation can be done
        #run trainingset through function to get k
        
        def select_k(data):
            n = nrow(data)
            #since rule of thumb k <= sqrt(n)
            #initializing variables
            best_k <- 1
            min_error <- NaN
            for i in 1 to sqrt(n):
                run knn function (data, k = i, metric, estimator) to output predicted labels
                compute min_squared_error between prediction and actual labels
                if min_error == NaN or min_squared_error < min_error:
                    best_k <- i
                    min_error <- min_squared_error
                else:
                    Next i
            return best_k    
        
        #Try knn using best_k with testingset?
</pre>

Q3. For the k-nearest regression. What happends when k = n. Where n is equal to the training size.

<pre> average/mode of all the training data points label value is the final predicted value for any intput into the model </pre>

Q4. Define a function that takes a 1-d numpy array, a parameter k, and a number p.
The function returns an estimate equal to the mean of the closest k points to the number p

In [4]:
def k_neighbor(input_data, k, p):
    """Returns the k-neighbor estimate for p using data input_data.
    
    Keyword arguments:
    input_data -- numpy array of all the data
    k -- Number of k
    p -- input values
    
    If you make assumptions please explain them in the comments. i.e. tie breaking strategy.
    """
    
    #YOUR CODE HERE
    distance_data = np.array(np.absolute(input_data - p))
    #print(distance_data)
    
    input_data_enhanced = pd.DataFrame({'data' : pd.Series(input_data, index=[x for x in range(0,15)]),
                                        'dist' : pd.Series(distance_data, index=[x for x in range(0,15)])})
    #print(input_data_enhanced.dtypes)
    #print(input_data_enhanced.describe())
    #print('original data and distances:')
    #print(input_data_enhanced.sort_values('dist',ascending = True))
    
    #in case of ties: random - first instance selected
    input_data_enhanced = input_data_enhanced.sort_values('dist',ascending = True)
    #print(input_data_enhanced)
    knn_mean = input_data_enhanced[:k]['data'].mean() 
    
    return knn_mean
    
    #return answer

In [5]:

data = np.array([1,3,4,5,7,8,11,12,13,15,19,24,25,29,40])
print(k_neighbor(input_data=data, k=3, p=5)==((5+4+3)/3)) 
print(k_neighbor(input_data=data, k=2, p=15)==((15+13)/2)) 
print(k_neighbor(input_data=data, k=3, p=25)==((24+25+29)/3))
print(k_neighbor(input_data=data, k=1, p=55)== 40)
print(k_neighbor(input_data=data, k=3, p=55)==((25+29+40)/3))
print(k_neighbor(input_data=data, k=10, p=55)==(np.mean([8,11,12,13,15,19,24,25,29,40])))

True
True
True
True
True
True


In [6]:
#Evaluate
data = np.array([1,3,4,5,7,8,11,12,13,15,19,24,25,29,40])
print(k_neighbor(input_data=data, k=3, p=5))
print(k_neighbor(input_data=data, k=2, p=15))
print(k_neighbor(input_data=data, k=3, p=25)) 
print(k_neighbor(input_data=data, k=1, p=55))
print(k_neighbor(input_data=data, k=3, p=55))
print(k_neighbor(input_data=data, k=10, p=55))

4.0
14.0
26.0
40.0
31.333333333333332
19.6


In [ ]:
#Enter your observations and comments here
#tie breaking strategy should differ based on the use case
#tried averaging results first by distance then taking the k nearest neighbour but this assumes 
#neighbours with varying distances are weighted the same
#also this method would have an issue since >=k points from the dataset would be considered in the calculation

Q5. Similar to Q4 but for the n dimentional case. 

In [128]:
def l1_norm(a,b):
    #takes in array a and b and computes L1 norm
    #Returns the l1 norm (a,b)
    l1_norm = np.mean(np.sum(np.absolute(np.subtract(np.array(a),np.array(b)))))
    return l1_norm

def l2_norm(a,b):
    """Returns the l2 norm (a,b)""" 
    l2_norm = np.sqrt(np.sum(np.square(np.subtract(np.array(a),np.array(b)))))
    return l2_norm
            
def k_neighbor_nd(input_data, k, p, metric='l1', mode='mean'):
    """Returns the k-neighbor estimate for p using data input_data.

    Keyword arguments:
    input_data -- numpy array of all the data
    k -- Number of k
    p -- input values
    metric -- l1 or l2. l1 norm or l2 norm https://en.wikipedia.org/wiki/Norm_(mathematics)
    mode -- estimator possible values = 'mean', 'median', 'max'
    
    Implement the l1 and l2 norms
    for mean, median and max, use np.mean, np.median and np.max
    """
    #Assuming the function returns a vector and not a scalar
    
    #YOUR CODE HERE
    input_data_enhanced = pd.DataFrame(input_data)
    if metric=='l2':
        input_data_enhanced['dist'] = input_data_enhanced.apply(lambda row: l2_norm(row,p), axis=1)
    elif metric=='l1':
        input_data_enhanced['dist'] = input_data_enhanced.apply(lambda row: l1_norm(row,p), axis=1)
    else:
        print('wrong estimator selected')
    #print(input_data_enhanced) #spot check
    
    #in case of ties: random - first instance selected
    input_data_enhanced = input_data_enhanced.sort_values('dist',ascending = True)
    #print(input_data_enhanced) #spot check
    #getting the k neighbours
    knn_value = input_data_enhanced[:k][input_data_enhanced.columns.difference(['dist'])]
        
    #return answer
    return eval('np.array(knn_value.'+ mode+'())')
   
    

In [129]:
data_4d = np.array([[4, 1, 2, 1], [1, 4, 2, 0], [3, 3, 1, 1], 
        [4, 0, 0, 0], [1, 2, 0, 0], [3, 4, 2, 3], 
        [2, 4, 4, 2], [2, 1, 4, 1], [3, 3, 2, 4], 
        [4, 3, 0, 4], [2, 2, 4, 0],[4, 3, 0, 2], 
        [4, 3, 0, 2], [0, 3, 4, 2]])

#print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l2', mode='mean'))

#print(pd.DataFrame(data_4d).iloc[0,])
#print(pd.DataFrame(np.array([[4, 1, 2, 1], [1, 4, 2, 0]])).values-pd.DataFrame(np.array([[2, 1, 4, 3]])).values)

[3.5 3.  0.5 1.5]


In [130]:
#Evaluate
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 2, 2, 4], metric='l1', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l1', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l1', mode='median'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 1, 4, 3], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=2, p=[4, 4, 0, 0], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 2, 2, 4], metric='l2', mode='max'))
print(k_neighbor_nd(input_data=data_4d, k=1, p=[2, 3, 3, 4], metric='l2', mode='mean'))
print(k_neighbor_nd(input_data=data_4d, k=3, p=[2, 3, 3, 4], metric='l2', mode='median'))

[2.         2.33333333 4.         1.        ]
[4. 3. 0. 2.]
[4 4 2 4]
[3. 3. 2. 4.]
[3. 4. 2. 3.]
[1.33333333 2.66666667 4.         1.66666667]
[3.5 3.  0.5 1.5]
[4 4 2 4]
[3. 3. 2. 4.]
[3. 4. 2. 3.]


In [ ]:
#Enter your observations and comments here

Q6[Optional]. Read the documentation on KNeighborsRegressor

http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
    
Explore the source code:
    https://github.com/scikit-learn/scikit-learn/blob/ef5cb84a/sklearn/neighbors/regression.py
        
How different it is from your implementation? How well can you follow the code? Did you learn something new?

<pre> implementation is similar </pre>